# SDOBenchmark Dataset

## Data Exploration 

### Dataset in a Nutshell

Lightweight dataset consisting of patches of SDO data consisting of patches centered around the active regions - some of which lead to flares. Prepared by people from FHNW/I4DS.<br>

Algorithm used to create the dataset from the raw data: see https://github.com/i4Ds/SDOBenchmark/blob/master/STRUCTURE.md <br>
- The target active region coordinates identified from TODO, calculated on the current image
- 512 x 512 patch cut out around the rotated coordinates (TODO??)
- Resulting image data clipped by predefined clipping ranges, scaled down to 256x256.
- Result saved to disk as 8-bit JPEG.

Samples consist of 
- 4 time steps with 
- 10 images per time step - 8 images from AIA, 2 images from HMI

### Download Data

Can be downloaded from https://i4ds.github.io/SDOBenchmark/ <br>

Two versions:
- data-full:    ~3730MB on disk (extracted)
- data-example:  ~340MB on disk (extracted)


In [ ]:
!mkdir -p ./data
!curl -L https://github.com/i4Ds/SDOBenchmark/archive/data-example.zip > data-example.zip
!unzip -q -o data-example.zip -d ./data
!rm -f data-example.zip

Install dependencies

- on Google Colab you will need to restart your runtime after this step
- when running this locally, make sure to first follow the setup instructions in the [README](../README.md)

In [ ]:
!pip install -U tqdm
!pip install -U torch torchvision
!pip install -U numpy
!pip install -U matplotlib 
!pip install -U Pillow
!pip install -U pandas

In [ ]:
# Data location

DATADIR = "./data/SDOBenchmark-data-example"



# download and extract it to DATADIR

# location of data
data_dir_train = "%s/training"%DATADIR
data_dir_test  = "%s/test"%DATADIR

#location of metadata
df_index_train = "%s/training/meta_data.csv"%DATADIR
df_index_test  = "%s/test/meta_data.csv"%DATADIR

### Dataset Structure

- Split into train and test, accordingly you find a $train$- and $test$-folder (8'336 train, 886 test samples).
- Different samples organized in separate subgroups, within a subgroup you will see samples observed close in time (distributed over several days):
    - $\verb|<subgroupid>|$
    - within the subgroup folder the data for __one sample__ can be found in a folder $\verb|<sampleid>|$ named with a date format: $\verb|YYYY-MM-DD-HH-mm-ss-v|$ (v: version ?? TODO)
- Each sample folder contains 40 images - 10 images for each of the 4 time steps.
- Images are named composed of a timestamp followed by the AIA band or HMI specification: $\verb|<timestamp>_<instrument_specifier>.jpg|$
    - timestamp: $\verb|YYYY\_MMTDDHHmmss|$
    - instrument specifier: 
        - for AIA: $\verb|94, 131, 171, 193, 211, 304, 335, 1700|$
        - for HMI: $\verb|continuum, magnetogram|$
- Label information provided in form of peak flux values:
    - one line per sample with $id$ ($\verb|<subgroupid>_<sampleid>|$), time range the images refer to ($start$, $end$) and $\verb|peak_flux|$ for the projection period.
    - the peak flux can be translated to a flare class label by comparing it with a threshold (for flare classes $\ge$ C we would use a threshold 1.0e-6, for flare classes $\ge$ M a threshold 1.0e-5). 

_Remark_: When splitting train into train and validate it is recommended to split along the subgroups, i.e. not split up the subgroups. 

The thresholding drives the distribution of the flare classes. When referring to FLARE as at least class C flare, we obtain 
- train: 346 flare and 7051 non-flare samples (TODO: these numbers are inconsistent) 
- test:  157 flare and  673 non-flare samples


#### Missing Data

Not for all samples we have data for all the 10 channels. This should be considered when loading and possibly processing and modeling the data.


### Data Access

In [ ]:
import os
from datetime import datetime, timedelta
from dateutil import parser
import random
import pandas as pd
from PIL import Image

AIA_94, AIA_131, AIA_171, AIA_193, AIA_211, AIA_304, AIA_335, AIA_1700 = "94", "131", "171", "193", "211", "304", "335", "1700"
HMI_CONT, HMI_MAGN = "continuum", "magnetogram"
ALL_CHANNELS = [AIA_94, AIA_131, AIA_171, AIA_193, AIA_211, AIA_304, AIA_335, AIA_1700, HMI_CONT, HMI_MAGN]

TIMESTEPS = [timedelta(), timedelta(hours=7), timedelta(hours=10, minutes=30), timedelta(hours=11, minutes=50)]

DATEFORMAT_IMGNAME = "%Y-%m-%dT%H%M%S"

class SDOBenchmarkIndex():
    
    def __init__(self, data_dir, df_index, channels=None, threshold=10e-6):
        self.data_dir = data_dir
        self.df_index = pd.read_csv(df_index)
        self.channels = self.channels_to_check(channels)
        self.threshold = threshold
        self.sample_dirs = []
        self.image_files = {}
        self.labels = {}

        print("Checking for channels: %s"%', '.join([ch for ch in self.channels]))
        print("Using threshold      : %9.8f"%self.threshold)
        
        for index, row in self.df_index.iterrows():
            groupid = row['id'][:5]
            dateid = row['id'][6:]
            group_dir = os.path.join(data_dir,groupid )
            sample_dir = os.path.join(group_dir, dateid)
            start_date = parser.parse(row['start'])

            if self.check_sample_dir(sample_dir, start_date, self.channels):
                imgs_steps = {}
                self.sample_dirs.append(sample_dir)
                self.labels[sample_dir] = self.create_binary_label(row['peak_flux'])
                for i in range(len(TIMESTEPS)):
                    imgs_channels = {}
                    for ch in self.channels:
                        _name = self.create_image_name(start_date, TIMESTEPS[i], ch) 
                        imgs_channels[ch] = os.path.join(sample_dir, _name)
                    imgs_steps[i] = imgs_channels
            
                self.image_files[sample_dir]=imgs_steps

        print("Found %i samples from which %i are flares."%(len(self.sample_dirs), sum(self.labels.values())))

    def channels_to_check(self, channels):
        if not channels:
            channels = ALL_CHANNELS.copy()
        return channels

    def create_image_name(self, base_date, delta, channel):
        img_date = base_date+delta
        img_name = "%s__%s.jpg"%(datetime.strftime(img_date, DATEFORMAT_IMGNAME),channel)
        return img_name
    
    def check_sample_dir(self, sample_dir, start, channels): #check if the directory contains all samples (no missing)        
        for delta in TIMESTEPS:
            for ch in channels:
                _name = self.create_image_name(start, delta, ch)
                _path = os.path.join(sample_dir, _name)
                if not os.path.exists(_path):
                    return False
        return True

    def create_binary_label(self, val):
        return int(val>self.threshold)


In [ ]:
train_index = SDOBenchmarkIndex(data_dir_train, df_index_train, channels=(HMI_MAGN,))

### Some Glances at the Data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
data = train_index

#### Load Data

In [ ]:
from PIL import Image

In [ ]:
sample = np.random.choice(list(data.image_files.keys()))
channel = HMI_MAGN
timestep = 0

image_path = data.image_files[sample][timestep][channel]
image = Image.open(image_path)
imgarray = np.asarray(image)

print(image.size, image.mode)
print(imgarray.min(), imgarray.max())
image

#### Plot Timesteps

In [ ]:
nsamples = 4
channel = HMI_MAGN

samples = [np.random.choice(list(data.image_files.keys())) for i in range(nsamples)]
nsteps = 4

plt.figure( figsize=(nsteps*3,nsamples*3) )
for i in range(nsamples):
    for j in range(nsteps):
        plt.subplot( nsamples, nsteps, nsamples*i+j+1 )
        image_file = data.image_files[samples[i]][j][channel]
        image = plt.imread(image_file)
        plt.title( "Sample %i, Step %i"%(i,j))
        plt.imshow( image[:,:], cmap="gray" )
        plt.axis('off')

#### Plot Large Mosaic (Timestep 0)

In [ ]:
n = 10
channel = HMI_MAGN
timestep = 0

plt.figure( figsize=(n*2,n*2) )
samples = [np.random.choice(list(data.image_files.keys())) for i in range(n*n)]

for i in range(n):
    for j in range(n):
        plt.subplot( n, n, n*i+j+1 )
        image_file = data.image_files[samples[n*i+j]][timestep][channel]
        image = plt.imread(image_file)
        plt.title( "Sample %i"%(i))
        plt.imshow( image[:,:], cmap="gray" )
        plt.axis('off')

#### Flux Distribution

In [ ]:
hist, bins, _ = plt.hist(data.df_index.peak_flux, bins=150)

## Modeling the Data

### PyTorch Data Loader (Dataset)

using SDOBenchmarkIndex 


In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF

from torch.utils.data import Dataset

In [ ]:
class SDOBenchmarkDataset(SDOBenchmarkIndex, nn.Module):
    def __init__(self, data_dir, df_index, channels=None, threshold=10e-6, size=256):
        super().__init__( data_dir, df_index, channels=None, threshold=10e-6)
        self.mean=0.4997
        self.std=0.1004
        self.size=size

    def transform (self, img_list):
        to_ten=transforms.ToTensor()
        for idx, img in enumerate(img_list):
            img_list[idx] = to_ten(img)
        
        resize=transforms.Resize(self.size)
        for idx, img in enumerate(img_list):
            img_list[idx]=resize(img)

        return img_list
        
    def create_channels(self, fold):
        l_all=[]
        for i in fold.keys():
            imgg=Image.open(fold[i]['magnetogram'])
            l_all.append(imgg)
        l_all =self.transform(l_all)
        for idx, img in enumerate(l_all):
            l_all[idx] = img.squeeze()
        return torch.stack(l_all)
        
    def __getitem__(self, index):
        
        fold=self.image_files[self.sample_dirs[index]]
        res_all=self.create_channels(fold)
        return res_all, self.labels[self.sample_dirs[index]]
       
    def __len__(self):
    
        return len(self.labels)

In [ ]:
train_index = SDOBenchmarkDataset(data_dir_train, df_index_train, channels=(HMI_MAGN,), size=64)
test_index = SDOBenchmarkDataset(data_dir_test, df_index_test, channels=(HMI_MAGN,), size=64)

# CNN

In [ ]:
# 4 channel CNN that output 2 probabilities for flare and for non-flare
import torch
import torch.nn as nn
import torch.nn.functional as F
class CNN(nn.Module):
    
    def __init__(self, z_dim, nf):
        
        super(CNN, self).__init__()
        colors_dim=4
        
        self.cnn_layers=nn.Sequential(
            #input is (nc)
            nn.Conv2d(colors_dim, nf, kernel_size=4, stride=2, padding=1, bias=False),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            # state size: (nf) *32 *32
            nn.Conv2d(nf, nf * 2, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(nf * 2),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            # state size: (nf * 4) * 16 * 16
            nn.Conv2d(nf * 2, nf * 4, kernel_size=4, stride=2, padding=1, bias=False),
            nn.BatchNorm2d(nf * 4),
            nn.LeakyReLU(negative_slope=0.2, inplace=True),
            # state size: (nf * 4) * 8* 8)
        )
        self.linear1=nn.Linear(32*4*8*8, 32*4*2)
        self.linear2=nn.Linear(32*4*2, 2)
        
        
    def forward (self, x):
        x=self.cnn_layers(x)
        x = x.view(x.size(0), -1)
        x=self.linear1(x)
        x = F.relu(x)
        x=self.linear2(x)
        x=F.softmax(x, dim=1)
        
        return x.squeeze()           

In [ ]:
from sklearn.utils import class_weight
import numpy as np
#creating dataloders 

train_loader=torch.utils.data.DataLoader(train_index, batch_size=64, pin_memory=True)
test_loader=torch.utils.data.DataLoader(test_index, batch_size=64)


#creating weights
cl_weights=class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(list(train_index.labels.values())), y=list(train_index.labels.values()))
cl_weights=torch.Tensor(cl_weights)

# using cuda if available for training and testing

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#weight to device
cl_weights=cl_weights.to(device)


#creating the model


z_dim=2
nf=32
model=CNN(z_dim, nf)
print(model)
#model to device
model.to(device)

#creating opimizer
lr=0.0001
optim=torch.optim.SGD(model.parameters(), lr)

#creating loss function
loss=nn.CrossEntropyLoss(cl_weights)

In [ ]:
loss_ar=[]
loss_test_ar=[]

y_out=torch.Tensor([])
y_all=torch.Tensor([])

y_out_test=torch.Tensor([])
y_all_test=torch.Tensor([])

epoch=10
for epochs in range (epoch):
    
    #creating sum of losses
    loss_all=0
    loss_all_test=0
    
    count=0
    count_test=0
    
    for x, y in train_loader:
        model.train()
        
        x=x.to(device)
        y=y.to(device)
        if epochs==epoch-1:
            y_all=torch.cat([y_all, y])
        
        #calculating output
        output=model(x)
        if epochs==epoch-1:
            y_out=torch.cat([y_out, output])
        
        #calculating loss
        l=loss(output, y.long())
        
        #backprop
        optim.zero_grad()
        l.backward()
        optim.step()
        # adding losses
        loss_all=loss_all + l
        count+=1
        
   
    
    #calculating output for test dataset
    model.eval()
    
    with torch.no_grad():
        for x, y in test_loader:
           
            #data to device
            
            x=x.to(device)
            y=y.to(device)
            if epochs==epoch-1:
                y_all_test=torch.cat([y_all_test, y])
        
            #calculating output
            
            output=model(x)
            if epochs==epoch-1:
                y_out_test=torch.cat([y_out_test, output])
        
            
            l=loss(output, y.long())
            
            #adding losses
            loss_all_test=loss_all_test + l
            count_test+=1
            
            
    #calculating losses
    loss_tr=float(loss_all)/count
    loss_ts=float(loss_all_test)/count_test
    
    loss_ar.append(loss_tr)
    loss_test_ar.append(loss_ts)
    
    print('epoch: {}, train_loss: {}, test_loss: {} \n'.format(epochs, loss_tr, loss_ts))

In [ ]:
#plotting learning curves

epochs=range(1, len(loss_ar)+1)
plt.plot(epochs, loss_ar, 'g', label='Training loss')
plt.plot(epochs, loss_test_ar, 'b', label='Test loss')
plt.title('training and test loss ')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix

tresh = 0.5 

cmat_test = confusion_matrix(y_all_test, np.array([y_out_test[i][0]<tresh for i in range(len(y_out_test))]))

In [ ]:
def tss(cmat):
    
    tp=cmat[1][1]
    fn=cmat[1][0]
    fp=cmat[0][1]
    tn=cmat[0][0]
    
    eps=1e-6
    
    tss=round(float(tp)/float(tp+fn+eps)-float(fp)/float(fp+tn+eps), 3)
    
    return tss

In [ ]:
print(cmat_test)

In [ ]:
print('tss: {}'.format(tss(cmat_test)))